# Practice Project: The System Bootloader 💻

## 🎓 Module 01 Summary
Before we start coding, let's recap the critical shifts from C to C++:

| Concept | C Approach | Modern C++ Approach |
| :--- | :--- | :--- |
| Compiler | gcc | g++ (links Standard Library) |
| Extensions | .h / .c | .hpp / .cpp |
| Build System | Make (Manual Flags) | CMake (Targets & Modules) |
| Dependencies | apt-get / Copy-paste | FetchContent / vcpkg |
| Printing | printf | std::cout |

---

## 🚀 The Mission: System Bootloader

You are writing the startup software for a server (or a spaceship, if you prefer). 
Your program needs to read a configuration file (config.json) and print the system status.

Since writing a JSON parser from scratch in C is painful (string parsing hell), we will use the industry-standard C++ library: nlohmann/json.

### 📋 Requirements

1.  Project Structure:
    * src/main.cpp
    * src/config_loader.cpp
    * include/config_loader.hpp
    * CMakeLists.txt
    * config.json
2.  Library: Use FetchContent in CMake to download nlohmann/json.
3.  Functionality:
    * Read system_name (string).
    * Read cpu_count (int).
    * Read enabled_modules (array of strings).
    * Print them using std::cout.

---

## 🛠️ Step 1: Project Layout

This module ships with a ready-to-use folder named `practice/`. It already contains `src/`, `include/`, `config.json`, and a starter `CMakeLists.txt`. If you want to recreate it from scratch, mirror this structure:

```text
practice/
├── CMakeLists.txt
├── CMakePresets.json
├── config.json
├── include/
│   └── config_loader.hpp
└── src/
    ├── config_loader.cpp
    └── main.cpp
```


## 🛠️ Step 2: The CMake Recipe

The `practice/CMakeLists.txt` uses `FetchContent` to pull the JSON library and links it into your app.

```cmake
cmake_minimum_required(VERSION 3.14)
project(Bootloader)

set(CMAKE_CXX_STANDARD 17)

# --- Dependency Management ---
include(FetchContent)
set(JSON_BuildTests OFF CACHE INTERNAL "")
FetchContent_Declare(
    json
    URL https://github.com/nlohmann/json/releases/download/v3.12.0/json.tar.xz
)
FetchContent_MakeAvailable(json)

# --- Project Setup ---
include_directories(include)

add_executable(boot_system 
    src/main.cpp 
    src/config_loader.cpp
)

target_include_directories(boot_system PRIVATE ${CMAKE_CURRENT_SOURCE_DIR}/include)
target_link_libraries(boot_system PRIVATE nlohmann_json::nlohmann_json)
```


## 🛠️ Step 3: The CMakePresets Recipe

```json
{
  "version": 3,
  "cmakeMinimumRequired": {
    "major": 3,
    "minor": 14,
    "patch": 0
  },
  "configurePresets": [
    {
      "name": "default",
      "displayName": "Default (Debug)",
      "description": "Configure the practice bootloader with Debug flags",
      "generator": "Unix Makefiles",
      "binaryDir": "${sourceDir}/build/default",
      "cacheVariables": {
        "CMAKE_BUILD_TYPE": "Debug",
        "CMAKE_EXPORT_COMPILE_COMMANDS": "ON"
      }
    }
  ],
  "buildPresets": [
    {
      "name": "default",
      "displayName": "Build Default",
      "configurePreset": "default"
    }
  ]
}
```

## 🛠️ Step 4: The C++ Code

Now, write the code (or inspect the provided reference). Hints:

### config.json
```json
{
  "system_name": "Orion-Server-01",
  "cpu_count": 16,
  "modules": ["Auth", "Database", "Cache"]
}
```

### include/config_loader.hpp
Define a namespace `Bootloader`. Declare `void load_config(const std::string& path);` inside it.

### src/config_loader.cpp
* Open the file with `std::ifstream`.
* Parse JSON with `nlohmann::json`.
* Print the fields (name, CPU count, modules).

### src/main.cpp
* Call `load_config("config.json")`.
* Wrap everything in `namespace Bootloader` to avoid name collisions.


---

## 🔎 Solution Reference

Try to implement it yourself first! If you get stuck on the syntax, check the blocks below.

In [1]:
// --- include/config_loader.hpp ---
#pragma once
#include <string>

namespace Bootloader {
    void load_config(const std::string& path);
}

In [ ]:
// --- src/config_loader.cpp ---
#include "config_loader.hpp"
#include <iostream>
#include <fstream>
#include <nlohmann/json.hpp>

// Alias for cleaner code
using json = nlohmann::json;

namespace Bootloader {
    void load_config(const std::string& path) {
        std::ifstream file(path);
        if (!file.is_open()) {
            std::cerr << "Error: Could not open config file!" << std::endl;
            return;
        }

        // Parse JSON directly from file stream
        json data = json::parse(file);

        // Accessing data (Notice how it looks like Python dictionaries)
        std::string name = data["system_name"];
        int cpu = data["cpu_count"];

        std::cout << "Booting System: " << name << std::endl;
        std::cout << "CPUs Detected:  " << cpu << std::endl;
        std::cout << "Modules: " << std::endl;

        // Range-based for loop (Modern C++)
        for (const auto& mod : data["modules"]) {
            std::cout << "  [+] " << mod << std::endl;
        }
    }
}

In [ ]:
// --- src/main.cpp ---
#include "config_loader.hpp"
#include <iostream>
#include <string>

int main() {
    const std::string config_path = "config.json";

    std::cout << "Loading config: " << config_path << std::endl;
    Bootloader::load_config(config_path);

    return 0;
}

## 🧪 Run It

Pick any of these workflows for `practice/`:

1) **CMake preset + make (recommended)**
```bash
cd 01_From_C_to_Cpp/practice
cmake --preset default        # configure + generate Makefiles
make -C build/default         # build
./build/default/boot_system  # run (config.json should be in cwd)
```

2) **VS Code + CMake Tools**
* In VS Code, select the `default` configure preset for `practice`.
* Run **Build** → it will call `make` with the preset.
* Run/Debug target `boot_system`. Ensure `config.json` is alongside the binary or your working directory.

3) **Manual out-of-source (no presets)**
```bash
cd 01_From_C_to_Cpp/practice
mkdir -p build/manual && cd build/manual
cmake ..
make
./boot_system
```
